# Prediccion de Default en Prestamos


Para este proyecto utilizaremos un sample de los datos de Lending Club. La idea es predecir si cierto usuario cometera Default basado en informacion que la plataforma recolecta. Esto nos ayudara a mejorar la metodologia/pipeline de prestamo.


# Descripcion



Contiene los prestamos de esta plataforma:

    periodo 2007-2017Q3.
    887mil observaciones, sample de 100mil
    150 variables
    Target: loan status



# Objetivo

Realizar un ETL y un EDA

## ETL

0. Limpia los datos de tal manera que al final del ETL queden en formato `tidy`.
1. Asegurate de cargar y leer los datos
2. Crea una tabla donde se guarde el nombre de la columna y el tipo de dato: (`column_name`,   `type`).
3. Asegurate de pensar cual es el tipo de dato correcto. Porque elejiste strig/object o float o int?. No hay respuestas incorrectas como tal, pero tienes que justificar tu decision.
4. Maneja missings o nans de la manera adecuada. Justifica cada decision







## EDA

0. Preparar lo datos para un pipeline de datos
1. Quitar columnas inservibles 
2. Imputar valores
3. Mantener replicabildiad y reproducibilidad

**No olvides anotar tus justificaciones en celdas para recordar cuando te toque explicarlo.** Puedes agregar el numero de celdas que necesites para poner tu explicacion y el codigo, solo manten la estructura.

# ETL

In [75]:
import pandas as pd
import numpy as np
import pickle
import json
pd.set_option('display.max_columns', None)

Vas a obtener 2 errores, solucionalo con los visto en clase.  
Tip: Se arreglan con argumentos adicionales de la funcion `read_csv`  
Documentacion: https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html 

In [25]:
# El archivo estaba comprimido. Hay que descomprimirlo primero
loans = pd.read_csv('https://github.com/sonder-art/fdd_prim_2023/blob/main/codigo/pandas/LoansData_sample.csv.gz?raw=true', compression = 'gzip', low_memory = False)
loans

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,0,38098114,NaN,15000.0,15000.0,15000.0,60 months,12.39,336.64,C,C1,MANAGEMENT,10+ years,RENT,78000.0,Source Verified,Dec-2014,Fully Paid,n,NaN,debt_consolidation,Debt consolidation,235xx,VA,12.03,0.0,Aug-1994,750.0,754.0,0.0,NaN,NaN,6.0,0.0,138008.0,29.0,17.0,w,0.00,0.00,17392.370000,17392.37,15000.00,2392.37,0.0,0.00,0.0000,Jun-2016,12017.81,NaN,Nov-2017,684.0,680.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,149140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,184500.0,NaN,NaN,NaN,5.0,29828.0,9525.0,4.7,0.0,0.0,103.0,244.0,1.0,1.0,0.0,47.0,NaN,NaN,NaN,0.0,1.0,4.0,1.0,2.0,8.0,5.0,9.0,4.0,6.0,0.0,0.0,0.0,4.0,100.0,0.0,0.0,0.0,196500.0,149140.0,10000.0,12000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,1,36805548,NaN,10400.0,10400.0,10400.0,36 months,6.99,321.08,A,A3,Truck Driver Delivery Personel,8 years,MORTGAGE,58000.0,Not Verified,Dec-2014,Charged Off,n,NaN,credit_card,Credit card refinancing,937xx,CA,14.92,0.0,Sep-1989,710.0,714.0,2.0,42.0,NaN,17.0,0.0,6133.0,31.6,36.0,w,0.00,0.00,6611.690000,6611.69,5217.75,872.67,0.0,521.27,93.8286,Aug-2016,321.08,NaN,Feb-2017,564.0,560.0,0.0,59.0,1.0,Individual,NaN,NaN,NaN,0.0,0.0,162110.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19400.0,NaN,NaN,NaN,7.0,9536.0,7599.0,41.5,0.0,0.0,76.0,290.0,1.0,1.0,1.0,5.0,42.0,1.0,42.0,4.0,6.0,9.0,7.0,18.0,2.0,14.0,32.0,9.0,17.0,0.0,0.0,0.0,4.0,83.3,14.3,0.0,0.0,179407.0,15030.0,13000.0,11325.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,

## Tabla (column_name, type)

Revisa el metodo pd.DataFrame.dtypes. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html 

In [26]:
column_types = loans.dtypes
column_types

Unnamed: 0                 int64
id                         int64
member_id                float64
loan_amnt                float64
funded_amnt              float64
                          ...   
settlement_status         object
settlement_date           object
settlement_amount        float64
settlement_percentage    float64
settlement_term          float64
Length: 151, dtype: object

## Cargar descripcion de columnas

La siguiente tabla tiene una descripcion del significado de cada columna

In [27]:


datos_dict = pd.read_excel(
    'https://resources.lendingclub.com/LCDataDictionary.xlsx')
datos_dict.columns = ['feature', 'description']


In [28]:
datos_dict

,feature,description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...
...,...,...
148,settlement_amount,The loan amount that the borrower has agreed t...
149,settlement_percentage,The settlement amount as a percentage of the p...
150,settlement_term,The number of months that the borrower will be...
151,NaN,NaN


### Pickle

Crea codigo para **guardar** y **cargar** el DataFrame de `datos_dict` creada en las celdas anteriores en formato **pickle**

In [29]:
# Codigo para 
# Como el pickle no es human-readable, se guarda en binario. Por eso se usa 'wb' (write-binary)
with open('datos_dict.pkl', 'wb') as file:
    pickle.dump(datos_dict, file)

In [30]:
# Codigo para cargar
datos_dict_from_pickle = None
# 'rb' = Read-binary. La razón es la misma que más arriba
with open('datos_dict.pkl', 'rb') as file:
    datos_dict_from_pickle = pickle.load(file)
print(datos_dict_from_pickle)

                   feature                                        description
0           acc_now_delinq  The number of accounts on which the borrower i...
1     acc_open_past_24mths         Number of trades opened in past 24 months.
2               addr_state  The state provided by the borrower in the loan...
3                 all_util              Balance to credit limit on all trades
4               annual_inc  The self-reported annual income provided by th...
..                     ...                                                ...
148      settlement_amount  The loan amount that the borrower has agreed t...
149  settlement_percentage  The settlement amount as a percentage of the p...
150        settlement_term  The number of months that the borrower will be...
151                    NaN                                                NaN
152                    NaN  * Employer Title replaces Employer Name for al...

[153 rows x 2 columns]


## Tipos de Datos

Realiza las transformaciones o casteos (casting) que creas necesarios a tus datos de tal manera que el typo de dato sea adecuado. Al terminar recrea la tabla `column_types` con los nuevos tipos.

No olvides anotar tus justificaciones para recordar cuando te toque explicarlo.

In [31]:
# Manejos de tipos 1: Parseos a fechas
# Todas las fechas del documento pueden tener el mismo formato: Mes-año. Hay que normalizar para poder usar formato de fecha

# Empezamos por una lista de los nombres de las columnas que contienen fechas
columnas_fechas = ["issue_d", "earliest_cr_line", "last_pymnt_d", "next_pymnt_d", "hardship_start_date", 
                   "hardship_end_date", "payment_plan_start_date", "settlement_date", "debt_settlement_flag_date",]

# Ahora hacemos el proceso de normalización:
for columna in columnas_fechas:
    # Cambiamos los strings a letras minusculas
    loans[columna] = loans[columna].str.lower()
    
    # Detectamos el formato de fecha y cambiamos el tipo de dato. Así aparecerá en formato aaaa-mm-dd
    loans[columna] = loans[columna].apply(lambda x: pd.to_datetime(x, format='%b-%y' if len(str(x)) == 6 else '%b-%Y', errors='coerce'))
    
    # Cambiamos el formato para que aparezca como aaaa-mm, sin día
    loans[columna] = loans[columna].apply(lambda x: x.to_period('M') if not pd.isnull(x) else x)

# Con esto quedó guardado en formato de fecha, específicamente period[M]

In [32]:
# Manejos de tipos 2: Parseos a int32
'''
    La mayoría de los datos que podemos convertir a enteros tienen una característica: la descripcion de la columna 
    refiere a numeros enteros no negativos y las columnas no contienen NaN's. Hay algunas columnas que si refieren a numeros enteros 
    pero que al contener Nan's no se puede hacer el casteo a int64.
'''

# Los datos que podemos castear directamente a enteros (por su descripción y por cómo vienen en el csv) son los siguientes:
columnas_enteros = ["delinq_2yrs", "inq_last_6mths", "open_acc", "pub_rec", "collections_12_mths_ex_med", 
                    "policy_code", "acc_now_delinq", "tot_coll_amt", "acc_open_past_24mths", "chargeoff_within_12_mths", 
                    "delinq_amnt", "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
                    "num_accts_ever_120_pd", "num_actv_bc_tl", "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", 
                    "num_il_tl", "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0", "num_sats", 
                    "num_tl_30dpd", "num_tl_90g_dpd_24m", "num_tl_op_past_12m", "pub_rec_bankruptcies", "tax_liens",]

# 'policy_code' solo puede tomar los valores 1 o 2. Lo cambie a int porque ocupa menos espacio en memoria que un object o que un float

# Convertimos esas columnas a enteros
loans[columnas_enteros] = loans[columnas_enteros].astype('int32')

# Otra columna que podemos convertir a enteros es 'term', pero requiere un manejo especial.
# 'term' contiene un valor entero de meses junto con la palabra 'months' o 'month'. Tenemos que quitar esos strings y luego parsear
loans["term"] = loans["term"].str.replace(r'\D', '', regex=True).astype('int32')


In [33]:
# Manejos de tipos 3: Parseos a float32

"""
    'emp_length' puede contener valores de 0 a 10. 0 es para menos de un año y 10 es para v10 años o mas
    Ahora, Cada celda aparece originalmente con un numero al principio y despues la palabra 'years'. Vamos a hacer lo mismo
    que con 'month'. Adicionalmente, tenemos que manejar que originalmente aparecen '10+' para 10 o mas y '<1' en lugar de 0.
    Como hay valores vacíos, quedará en float, pero ocupa menos memoria que un string.
"""

# Reemplazar "<1 year" por 0 y "10+ years" por 10
loans["emp_length"] = loans["emp_length"].replace({"< 1 year": "0", "10+ years": "10"}, regex=True)

# Extraer números de la columna
loans["emp_length"] = loans["emp_length"].str.extract(r'(\d+)').astype('float32')

In [34]:
# Manejos de tipos 4: Parseo a String/Object

"""
    El id está en formato de int64. Esto es mas ligero, pero podria ser que el formato del id significa algo.
    Por ejemplo, los primeros digitos podrían significar el "Libro" en el que se registraron, algo parecido al CURP.
    Por ello lo convertiré a Strings, ya que en Strings se pueden aplicar regex para buscar características específicas.
    Además, si se quisiera regresar a int64 es muy facil, ya que no se le agregaría ningun caracter
"""
loans["id"] = loans["id"].astype(str)

In [35]:
# Manejos de tipos 4: Parseo a boolean

"""
    La columna 'pymnt_plan' indica si se implementó un plan de pago de prestamo o no. 
    Por la naturaleza de la columna, podemos poner False si no hay un plan (indicado con 'n') o True en caso de si haber plan.
    Todas las columnas contienen n, pero podía ser el caso de que contengan otro valor. Por esto convertire la columna a booleana.
    Esto hará mucho mas compacta la columna, y ocupará menos espacio en memoria.
"""

# Creo un diccionario de mapeo booleano
mapeo_booleano = {'n': False, 'y': True}

# Aplico el mapeo a la columna
loans["pymnt_plan"] = loans["pymnt_plan"].map(mapeo_booleano)

In [36]:
column_types = loans.dtypes
column_types

Unnamed: 0                   int64
id                          object
member_id                  float64
loan_amnt                  float64
funded_amnt                float64
                           ...    
settlement_status           object
settlement_date          period[M]
settlement_amount          float64
settlement_percentage      float64
settlement_term            float64
Length: 151, dtype: object

## Eliminación de columnas inservibles

In [37]:
"""
    En primer lugar, las columnas que tienen terminación '_joint' sirven para 'co-borrowers' y todas esas celdas están vacías.
    Esto quiere decir que no hay co_borrowers. Un caso similar ocurre con las columnas que inician con 'sec_app_', 
    que se refieren a 'secondary applicants', pero de igual forma todas esas celdas están vacías, por lo cual las eliminaré.
"""
# Lista de columnas que terminan con '_joints'
columnas_joins = [col for col in loans.columns if col.endswith('_joint')]

# Lista de columnas que inician con 'sec_app_'
columnas_sec = [col for col in loans.columns if col.startswith('sec_app_')]

# Elimina las columnas
loans = loans.drop(columnas_joins, axis = 1)
loans = loans.drop(columnas_sec, axis = 1)

In [38]:
"""
    Otras columnas que no proporcionan información son la columna "Unnamed: 0" (que únicamente contiene los numeros del 1 al 100000) y 'member_id',
    ya que no contiene ningún elemento y con el 'id' basta para distinguir entre observaciones
"""

loans = loans.drop(["Unnamed: 0", "member_id"], axis = 1)

In [39]:
"""
    Las columnas 'loan_amnt', 'funded_amnt' y 'funded_amnt_inv' contienen los mismos datos. La descripcion de las columnas nos dice que 
    'loan_amnt' es cuánto solicita la persona que pide el préstamo y que esa cantidad la pueden modificar según lo que decida el prestamista.
    
    'funded_amnt' refiere a la cantidad de dinero que se acepta dar y 'funded_amnt_inv' es la cantidad de dinero que aceptan dar los inversionistas.
    Como estas dos columnas son identicas, se da a entender que todo el dinero viene de inversionistas y no vale la pena tener los datos repetidos.
    
    Dejaré 'loan_amnt' y  'funded_amnt_inv'
"""
loans = loans.drop(["funded_amnt"], axis = 1)

In [40]:
"""
    Algo muy similar pasa con las columnas 'purpose' y 'title'. 'purpose' es el motivo que el solicitante del préstamo da y 'title' es la 
    forma en que lo guarda el prestamista. Los contenidos de ambas columnas solo distan por algunos caracteres, pero parece que el prestamista
    escribió el 'purpose' en 'title'. Me quedaré con 'title', que es el título con el que se guardó el préstamo.
"""
loans = loans.drop(['purpose'], axis = 1)

In [41]:
"""
    Lo mismo que con las anteriores pasa con 'funded_amnt' y 'funded_amnt_inv' ocurre con 'out_prncp' y 'out_prncp_inv', así como 
    con 'total_pymnt' y 'total_pymnt_inv'. Me quedaré con 'out_prncp_inv' y 'total_pymnt_inv'
"""
loans = loans.drop(['out_prncp', 'total_pymnt'], axis = 1)

In [42]:
# Con la información de la columna 'grade' tengo suficiente para el analisis que haré
loans = loans.drop("sub_grade", axis = 1)


## Manejo de NaNs o missings

Maneja los datos de tipos missing. Elije una estrategia adecuada dependiendo del tipo de dato que le asignaste a la columna.


In [43]:
# Elimino las columnas que tengan un 97% o más de NaN's. Esto facilitará el análisis más adelante

loans = loans.dropna(thresh = len(loans)*.97, axis = 1)

# Muestra cómo quedó la tabla loans
loans

,id,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp_inv,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,38098114,15000.0,15000.0,60,12.39,336.64,C,RENT,78000.0,Source Verified,2014-12,Fully Paid,False,Debt consolidation,235xx,VA,12.03,0,1994-08,750.0,754.0,0,6,0,138008.0,29.0,17.0,w,0.00,17392.37,15000.00,2392.37,0.0,0.00,0.0000,2016-06,12017.81,Nov-2017,684.0,680.0,0,1,Individual,0,0,149140.0,184500.0,5,29828.0,9525.0,4.7,0,0,244,1,1,0,47.0,0,1,4,1,2,8,5,9,4,6,0.0,0,0,4,100.0,0.0,0,0,196500.0,149140.0,10000.0,12000.0,N,Cash,N
1,36805548,10400.0,10400.0,36,6.99,321.08,A,MORTGAGE,58000.0,Not Verified,2014-12,Charged Off,False,Credit card refinancing,937xx,CA,14.92,0,1989-09,710.0,714.0,2,17,0,6133.0,31.6,36.0,w,0.00,6611.69,5217.75,872.67,0.0,521.27,93.8286,2016-08,321.08,Feb-2017,564.0,560.0,0,1,Individual,0,0,162110.0,19400.0,7,9536.0,7599.0,41.5,0,0,290,1,1,1,5.0,4,6,9,7,18,2,14,32,9,17,0.0,0,0,4,83.3,14.3,0,0,179407.0,15030.0,13000.0,11325.0,N,Cash,N
2,37842129,21425.0,21425.0,60,15.59,516.36,D,RENT,63800.0,Source Verified,2014-12,Fully Paid,False,Credit card refinancing,658xx,MO,18.49,0,2003-08,685.0,689.0,0,10,0,16374.0,76.2,35.0,w,0.00,25512.20,21425.00,4087.20,0.0,0.00,0.0000,2016-05,17813.19,Dec-2017,704.0,700.0,0,1,Individual,0,0,42315.0,21500.0,4,4232.0,324.0,97.8,0,0,136,7,7,0,7.0,1,3,4,3,12,16,5,18,4,10,0.0,0,0,2,91.4,100.0,0,0,57073.0,42315.0,15000.0,35573.0,N,Cash,N
3,37612354,12800.0,12800.0,60,17.14,319.08,D,MORTGAGE,125000.0,Verified,2014-12,Current,False,Car financing,953xx,CA,8.31,1,2000-10,665.0,669.0,0,8,0,5753.0,100.9,13.0,w,6673.54,11207.67,6126.46,5081.21,0.0,0.00,0.0000,2017-12,319.08,Dec-2017,629.0,625.0,0,1,Individual,0,0,261815.0,5700.0,2,32727.0,0.0,103.2,0,0,170,21,16,5,21.0,1,3,5,3,5,1,5,7,5,8,0.0,0,0,0,76.9,100.0,0,0,368700.0,18007.0,4400.0,18000.0,N,Cash,N
4,37662224,7650.0,7650.0,36,13.66,260.20,C,RENT,50000.0,Source Verified,2014-12,Charged Off,False,Debt consolidation,850xx,AZ,34.81,0,2002-08,685.0,689.0,1,11,0,16822.0,91.9,20.0,f,0.00,2281.98,704.38,339.61,0.0,1237.99,222.8382,2015-08,17.70,Oct-2016,559.0,555.0,0,1,Individual,0,0,64426.0,18300.0,6,5857.0,332.0,93.2,0,0,148,8,8,0,17.0,0,1,4,1,4,12,4,8,4,11,0.0,0,0,2,100.0,100.0,0,0,82331.0,64426.0,4900.0,64031.0,N,Cash,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,22454240,8400.0,8400.0,36,9.17,267.79,B,MORTGAGE,91000.0,Source Verified,2014-08,Fully Paid,False,Debt consolidation,606xx,IL,7.78,1,1990-08,705.0,709.0,1,6,1,8495.0,55.5,16.0,w,0.00,9640.15,8400.00,1240.15,0.0,0.00,0.0000,2017-08,267.50,Dec-2017,714.0,710.0,0,1,Individual,0,0,152181.0,15300.0,2,2536

Crea codigo para **guardar** y **cargar** un archivo JSON en el que se guarde la `estrategia` y `valor` que utilizaste para **imputar**. Por ejemplo: Si hay una columna que se llama `columna 3` y utilizaste la estrategia de imputacion de media, y existe otra llamada `columna 4` y  elegiste la palabra 'missing' el JSON debera contener:  
  
 `{'columna 3':{'estrategia':'mean', 'valor':3.4}, 'columna 4':{'estrategia':'identificador', 'valor':'missing'}}`  

 De tal manera que para cada columna que tenga un metodo de imputacion apunte a otro diccionario donde el **key** `estrategia` describa de manera sencilla el metodo, y el **key** `valor` el valor usado. En general:   
 `{'nombre de la columna':{'estrategia':'descripcion de estrategia', 'valor':'valor utilizado'}}`. 
 

De utilizar mas de un metodo puedes anidarlos en una lista  
  `[{...},{...}]`.  

Incluso si la columna utilizada no sufrio imputacion, es necesario que la agregues al JSON.

La idea es que cualquier otra persona pueda cargar el el archivo JSON con tu funcion, entender que hiciste y replicarlo facilmente. No existe solo una respuesta correcta, pero tendras que justificar y explicar tus deciciones.

### Imputacion

In [73]:
# Crearé un subconjunto del dataframe loans con las columnas que utilizaré

sub_loans = loans[['id', 'loan_amnt', 'annual_inc', 'grade', 'acc_now_delinq', 'percent_bc_gt_75']]
sub_loans

,id,loan_amnt,annual_inc,grade,acc_now_delinq,percent_bc_gt_75
0,38098114,15000.0,78000.0,C,0,0.0
1,36805548,10400.0,58000.0,A,0,14.3
2,37842129,21425.0,63800.0,D,0,100.0
3,37612354,12800.0,125000.0,D,0,100.0
4,37662224,7650.0,50000.0,C,0,100.0
...,...,...,...,...,...,...
99995,22454240,8400.0,91000.0,B,0,50.0
99996,11396920,10000.0,53133.0,C,0,11.1
99997,8556176,30000.0,78000.0,E,0,100.0
99998,24023408,8475.0,52000.0,F,0,100.0


In [74]:
"""
    Filtraré según los siguientes parámetros:
        - Personas que tengan un 'acc_now_delinq' > 0
        - Personas que tengan el 70% de sus tarjetas de crédito estén al 80% de su límite
        - Personas que tengn un 'grade' de C o D
        - Personas que el 'loan_amnt' sea mayor o igual al 25% de su 'annual_inc'
"""
resultado = sub_loans[sub_loans['acc_now_delinq']>0]
resultado = resultado[resultado['percent_bc_gt_75']>=80]
resultado = resultado[(resultado['grade'] == 'C') | (resultado['grade'] == 'D')]
resultado = resultado[resultado['loan_amnt'] >= 0.25 * resultado['annual_inc']]
resultado

,id,loan_amnt,annual_inc,grade,acc_now_delinq,percent_bc_gt_75
6293,34503335,22400.0,75000.0,D,1,100.0
12161,35883635,5000.0,20000.0,C,1,100.0
12667,35673982,20000.0,70753.0,D,1,100.0
13827,35893618,12250.0,41000.0,D,1,100.0
25938,34433190,20000.0,60000.0,C,1,85.7
26067,34312086,35000.0,91511.0,C,1,100.0
39378,32149427,18000.0,36000.0,D,1,100.0
40132,32129546,25000.0,82062.5,C,1,100.0
44128,31517216,32000.0,90000.0,D,1,100.0
53873,30355424,9025.0,30000.0,D,1,100.0


### Codigo para salvar y cargar JSONs

In [76]:
# Salvar en un json
procedimiento = {
    'loan_amnt':{'estrategia': 'Filtrar los elementos cuyo préstamo sea de al menos 25% De su ingreso anual', 
                 'valor':'>=.25 del annual_inc'},
    'annual_inc':{'estrategia': 'Obtener el valor para usarlo con loan_amnt', 'valor':'Directo de la tabla'},
    'grade':{'estrategia': 'Filtrar los elementos que cumplan cierta caracteristica', 'valor':'C o D'},
    'acc_now_delinq':{'estrategia': 'Filtrar los elementos que tengan un valor positivo', 'valor':'>0'},
    'percent_bc_gt_75':{'estrategia': 'Filtar los elementos que el 80% De sus tarjetas de crédito estén al 75% Del límite de crédito', 
                        'valor':'>=80'}
}

with open("procedimiento.json", 'w') as archivo_json:
    json.dump(procedimiento, archivo_json)

In [79]:
# Código para cargar el archivo json en forma de diccionario

with open("procedimiento.json", 'r') as archivo_json:
    diccionario_cargado = json.load(archivo_json)
    
diccionario_cargado

{'loan_amnt': {'estrategia': 'Filtrar los elementos cuyo préstamo sea de al menos 25% De su ingreso anual',
  'valor': '>=.25 del annual_inc'},
 'annual_inc': {'estrategia': 'Obtener el valor para usarlo con loan_amnt',
  'valor': 'Directo de la tabla'},
 'grade': {'estrategia': 'Filtrar los elementos que cumplan cierta caracteristica',
  'valor': 'C o D'},
 'acc_now_delinq': {'estrategia': 'Filtrar los elementos que tengan un valor positivo',
  'valor': '>0'},
 'percent_bc_gt_75': {'estrategia': 'Filtar los elementos que el 80% De sus tarjetas de crédito estén al 75% Del límite de crédito',
  'valor': '>=80'}}